# Marine Computer Science Lab - ML & AI project

### Importing packages and loading dataset

In [ ]:
from utilities import *

# Reading data from a CSV file hosted on Github
raw_df = pd.read_csv('https://raw.githubusercontent.com/edv1hu/TMR4345-Project-ML/main/data.csv')
# Setting the default style for Seaborn plots to darkgrid
# and increasing the font size


In [ ]:
# Checking the dataset
raw_df.head()

In [ ]:
raw_df.Trip_no.value_counts()
#raw_df.columns

# Task 1: Data Exploration & Visualization 

## a) Statistical properties

In [ ]:
# Generate descriptive statistics for the data
raw_df.describe()

## b) Histograms with its mean, median and mode

In [ ]:
plot_histograms(raw_df)

### b) Is the data symmetrically distributed?

From the skew function below we see that *n* and *Trip_no* has the lowest skewness, but show a clear pattern for each trip and is far from normally distributed.

*calm_r, wave_dir, wind_V *and* wind_V_long* have skewness under 0.25, and show some signs of symmetry. *wind_V* and *wind_V_long* also show signs of being somewhat normally distributed.

The rest of the variables isn't very symmetrical based on the skewness, but we can see some small signs visually. *Hs* is expected to be Rayleigh distributed and therefore not be very symmetrical, but would probably be more symmetrical after removing some/the outliers. Tp and added_r show signs of symmetry around the main peak.  

In [ ]:
# Calculate the skewness of each column in the dataframe
raw_df.skew().round(3)

## c) Abnormal, invalid or unrealistic values 

- The peak wave period, Tp, shows unrealistic values. Especially, the high count of $Tp \approx 100 s$ is noticeable.We know the usual range for regular waves is between 5s and 25s. To have some safety factor, I've removed values over 30s. The lower values seem ok, as the lowest Tp is 2.84s. 

- The unrealistic high count of low *Hs*-values ($Hs \approx 0$) corresponds to the high *Tp*-values and is also removed by removing all the rows with Tp over 30s.
  
- The high count of ***wind_dir, wind_V_trans*** $\approx 0$ and the abnormal peaks for *wind_V* and *wind_V_long* (seen by the modes) also corresponds with the rows mentioned above, and it seems ok to remove the whole rows with unrealistically high *Tp*-values.

- I considered removing trip 6 and 11 due to few datapoints, but the datapoints are fine, so it's more valuable to keep as many datapoints as possible
  

In [ ]:
# Identify rows in the DataFrame where the 'Tp' is greater than 30
# Saving/copying these as outlies
outliers = raw_df.loc[(raw_df.Tp >30)].copy()

# Create a new DataFrame 'df' without the outliers
df = raw_df.drop(outliers.index)

## d) Time-series with erroneous samples

These samples are erroneous since the Tp is unrealistically high and Hs very low. It also cleans up discrepancies with the wind-variables. 

I've also noticed 2 peaks in the timeseries of added_r and tot_r that I don't understand completely. Every trip seems to have the highest peaks around the same time, around 2300 and 9000 datapoints into each trip. 


In [ ]:
plot_grouped_scatterplots(df, 'Trip_no', outliers)


# Task 2: Data Analysis & Correlation Study

## a) Heatmap with Pearson's correlation

In [ ]:
# Set fontsize of the plot
sns.set(font_scale=1)

# Create a heatmap of the correlation matrix
fig = plt.figure(figsize=(15,12))
fig = sns.heatmap(data = df.corr(), cmap = 'Blues', annot = True, linewidths=0.5)

## b) Cross-plots

In [ ]:
# Create cross-plots with kernel density estimates on the diagonal
sns.set(font_scale = 2)
sns.pairplot(data=df , diag_kind='kde')

In [ ]:
sns.set(font_scale = 1)
# plot wind_V_long,Hs vs power/V, hue=n
sns.scatterplot(x =df.V, y = df.wind_V_long, hue = df.n )

# coloring the plots with the most important variable of the dataset gives another dimention and makes it easier
# to understand the dataset.
# Here we see a clear difference in rpm for similar speed at different longitudinal wind speeds. High wind_V_long 
# results in higher n relative to the same V with lower wind_V_long. This is because of the added resistance.

### b) Variable correlations 

**Strong (S)/ Weak (W), Positive (P)/ Negative (N) , Linear (L)/Non-linear (NL)** <br>
Eks. S,P,L = Strong, Positive, Linear 

Some relations are neither positive or negative, ex. Hs vs P, but I've understood the task to choose one of them either way. So I've based my answers on the signs in the heatmap/correlations.

Weak/Strong is based on the absolute correlation value being over or under 0.5. This is a little bit wrong as the correlation only is for linear correlation, but it should be good enough for this task as you said

Linear/non-linear is based on visual interpretation, but flat lines or bulks of datapoints that show little to no signs of correlations is set to Non-linear. Had a hard time determining the results because of the the different trips, maybe I it would be easier if I looked at a pairplot for only 1 trip?


|       | P  | Tp  | V   | added_r | calm_r | n  | tot_r | wave_dir | wind_V | wind_dir | wind_V_long | wind_V_trans | Trip_no |
| :---: | ---| --- | --- | --- | --- | --- | --- |--- | --- | --- | --- |--- | --- | 
|**Hs** |W , N , NL|W, P, NL| W, N, NL| W, P, NL|W, N, NL|W, N, NL| W, P, NL|W, P, NL|W, P, NL|W, N, NL|W, N, NL| W, P, NL| W, P, NL| 
| **P** |    |W, N, NL|S, P, L| W, N, NL|S, P, NL|S, P, NL|S, P, NL|W, N, NL|W, P, NL|W, P, NL|W, P, NL|W, P, NL|S, P, NL|
| **Tp** |   |     |W, N, NL|W, P, NL|W, N, NL|W, N, NL|W, P, NL|W, P,  NL|W, N, NL|W, N, NL|W, N, NL|W, N, NL|W, P, NL|
| **V** |    |     |     |S,N, NL|S,P, NL|S,P, L|S,P, NL|W, N, NL|W, N, NL|W, P, NL|W, P, L|W, N, NL|S,P, NL|
| **added_r** | |     |     |     | S, N, L |W,N,  NL|S, P, L|W, P, NL|W, P, NL|W, P, NL|W, P, NL|W, P, NL|W, N, NL|
| **calm_r** | |     |     |     |     |S, P, NL|S, P, NL|W, N, NL|W, N, NL|W, P, NL|W, P, NL|W, N, NL|S, P, NL|
| **n** | |     |     |     |     |    |S, P, L|W, N, NL|W, P, NL|W, P, NL|W, P, NL|W, N, NL|S, P, L|
| **tot_r** | |     |     |     |     |    |      |W, P, NL|W, P, NL|W, P, NL|W, P, NL|W, P, NL| S, P, NL|   
| **wave_dir** | |     |     |     |     |    |      |      |S, P, NL|W, P, NL|S, P, NL|W, P, NL|W, P, NL|   
| **wind_V** | |     |     |     |     |    |      |      |           |S, P, NL|S, P, NL|W, P, NL|W, P, NL|   
| **wind_dir** | |     |     |     |     |    |      |      |           |        |S, P, NL|W, N, NL|W, P, NL|  
| **wind_V_long** | |     |     |     |     |    |      |      |           |        |          |W, P, NL|W, P, NL|  
| **wind_V_trans** | |     |     |     |     |    |      |      |           |        |          |            |W, P, NL|   

## c) Dependent and Independent variables & functional relationship 

|  Independent variables | Dependent variables | Functional relationship |
| :---: | :---: | --- |
| Hs | P | P = f(n, Trip_no, Tp, Hs, wave_dir, wind_V_long, wind_V_trans) |
| Tp | V | V = f(n, Trip_no, Tp, Hs, wave_dir, wind_V_long, wind_V_trans) |
|  n | added_r | added_r = f(Tp, Hs, wave_dir, wind_V_long, wind_V_trans) |
| wave_dir |  calm_r | calm_r = f(n, Trip_no) |
|  wind_V | tot_r | tot_r = f(n, Trip_no, Tp, Hs, wave_dir, wind_V_long, wind_V_trans) |
| wind_dir | 
| wind_V_long | 
| wind_V_trans |
| Trip_no (draft) |

In [ ]:
# Dictionary with dependent variables and functional relationships
functional_relationship = {
    'P' : ['n', 'Trip_no', 'Tp', 'Hs', 'wave_dir','wind_V_long', 'wind_V_trans'],
    'V' : ['n', 'Trip_no', 'Tp', 'Hs', 'wave_dir','wind_V_long', 'wind_V_trans'],
    'added_r'   : ['Tp', 'Hs', 'wave_dir','wind_V_long', 'wind_V_trans'],
    'calm_r'    : ['n', 'Trip_no'],
    'tot_r'     : ['n', 'Trip_no', 'Tp', 'Hs', 'wave_dir','wind_V_long', 'wind_V_trans']
} #Might neglecy trans_V

## d) Data good enough for regression analysis? 

Yes, the dataset should be large enough to provide a good representation of the underlying relationship between the independent and dependent variables. With 9400 rows per trip (except trip 6 and 11 with only 1500 rows), it could be considered as a large dataset. The dataset also has independent variables to represent the dependent variables, and the dataset is cleaned to reduce abnormal values.


It's important to have the most important independent variables so the dataset can make good predictions of the dependent variables. In this dataset it's especially important to have n, as it's the highest correlation to most of the dependant variables. From the "Ship's hydrodynamic model" we also see that our dataset has most of the necessary variables to determine Speed, power, etc. The variables we lack is: Current (longitudinal speed), Draft, trim and rudder angle. But in this dataset the trim is set to 0, draft is increased uniformly with each trip and all directions is relative to the ships heading. Therefor, we have enough information to make a good regression analysis. 

## e) Additional variables 

Draft - The trips have different loading, resulting in different resistance for approximately the same weather conditions and n. Later recieved information that Trip_no represent the draft as the draft is uniformly increased for each trip. Still, having the real draft could be valuable for a better understanding of the increased resistance and to improve the analysis.

Reynolds number, Froudes number - Thought about adding these, but they would mainly be another representation of V.

TRIM - Should be added as it affects the ships hydrodynamics, but in this case the trim = 0 for all trips

RUDDER ANGLE - It's important as it helps determine the hydrodynamic forces acting on the ships hull and the resulting changes in the ship motion. Also, if the wave/wind directions wasn't given relative to the ship heading, then if would be valuable to know the ship heading or use the rudder angle

CURRENT, LONGTUDINAL SPEED - As its another factor affecting the added resistance it should be used if its possible to get hold of the data, as it could further improve the model.

WAVE ENERGY - Also looked into the possebility to represent the wave forces in a longtudinal and transverse way, as it might be a better representation then only having Hs, Tp and the direction, but of course this wasn't straight forward. In theory, I believe it would improve the model. 

# Task 3: Regression Modeling

## a) Training, validation and test-set 

First, I've tried to split the data in a random 60/20/20 split, but this causes the three datasets to have near identical samples. For example, this makes RandomForests unrealistically accurate. 

Therefore, a split should be of continuous sections of time-series. So I've decided to use two regular trips (not from the extremes) as test-set, one regular trip as the validation-set and the rest as training set. This makes for a train/validation/test split of 68/11/21. 

In [ ]:
train_df, valid_df, test_df = divide_data(df, [3, 9], 7)


In [ ]:
plot_data_scatter(train_df, valid_df, test_df, 'n', 'P')

## b) Standardizing the data using mean and standard deviation of the training data

In [ ]:


std_df_train, std_df_val, std_df_test = standardization(train_df, valid_df, test_df)

## c) Linear regression models

In [ ]:
lasso_models = lasso_reg_models(functional_relationship, std_df_train, std_df_val, std_df_test)

In [ ]:
ridge_models = ridge_reg_models(functional_relationship, std_df_train, std_df_val, std_df_test)

In [ ]:
# models = lin_reg_models(functional_relationship, std_df_train, std_df_val, std_df_test)

In [ ]:
lasso_results = evaluate_models(lasso_models, functional_relationship, std_df_train, std_df_val, std_df_test)

#evaluate_models(ridge_models, functional_relationship, std_df_train, std_df_val, std_df_test)


## d) Non-linear regression models

In [ ]:
models_MLP = MLP_models(functional_relationship, std_df_train, std_df_val, std_df_test)

In [ ]:
MLP_results = evaluate_models(models_MLP, functional_relationship, std_df_train, std_df_val, std_df_test)

# Doesn't make too much sence to plot a correlation-plot for the non-linear model as I did for the linear model
# mention possible overfitting

## e) Linear vs non-linear models

### Is the linear approach good enough to perform the task? 
No, from my results the non-linear method does a much greater job. Especially, the validation set is a problem for the linear model. On the other hand, the MLP-models might be overfitting as it has very high training score, but on validation for P it has 0,16 R2-score, and 0,74 for tot_r. 

### Is it possible to further improve the results from the linear model?

Yes, it would be possible to add non-linear transformations or split the data into more linear models so the 'resulting' model would be a combination of more than 1 linear model. With increasing number of models it would get closer to a non-linear model.

In [ ]:
display(lasso_results, MLP_results)

## f) Plots of regressions residuals

In [ ]:
plot_residuals(lasso_models, functional_relationship, std_df_train,  std_df_val,std_df_test)

The residuals for the linear model shows a lot of non-linear relations, which makes sence for why it mostly performe quite bad. 

We also see that the model has trouble with 'following' each trip, making the models especially bad for some trips like Trip 7 / the validation set

In [ ]:
plot_residuals(models_MLP, functional_relationship, std_df_train, std_df_val, std_df_test)

The residual plots for the Non-linear models show more uniform plots with constant variance - which is good. But we still see som patterns. Especially Tp has a the biggest errors at the mean value as we also saw on the linear models.  

Also the residual plot of n for calm_r shows the trips as continous 'hyperbolas' or 'waves', maybe it indicates that the model has trouble with getting all the differences between the trips. Maybe I need more independent variables, or the model could be made with respect to each trip?

In the P-model we also see an outlier in all the plots from trip 11.

## Comparing different Non-linear models

In [ ]:
train_evaluate_NLmodels(functional_relationship, std_df_train, std_df_val, std_df_test)